In [165]:
import os
import pandas as pd
import sys
import json
import re

In [166]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [167]:
db_df = pd.read_csv('WikiSQL_wikisql_test_final.csv')

In [168]:
import pandas as pd

# Define the path to your JSONL file
jsonl_file_path = 'WikiSQL_data_test.tables.jsonl'

# Read the JSONL file into a Pandas DataFrame
df = pd.read_json(jsonl_file_path, lines=True)

In [169]:
db_df.drop(['phase','sel','conds','agg'],axis=1,inplace=True)

In [170]:
db_df['fileName'] = 'TBA'

In [171]:
db_df['filePath'] = 'TBA'

In [172]:
db_df['result'] = 'Not available'

In [173]:
db_df.rename(columns={'question':'TQL','sql':'SQL','table_tableid':'db_id'},inplace=True)

In [174]:
df = df[['id','header','types','name']]

In [175]:
merged_df = db_df.merge(df, left_on='table_id', right_on='id', how='inner')


In [176]:
# Function to replace 'colX' values in SQL with values from the header column
def replace_col_values(sql, header):
    def replace_col(match):
        col_number = int(match.group(1))  # Extract the X from colX
        return header[col_number - 1]  # Use X to index the header list (0-based)

    return re.sub(r'col(\d+)', replace_col, sql)

# Apply the function to each row in the dataframe
merged_df['SQL'] = merged_df.apply(lambda row: replace_col_values(row['SQL'], row['header']), axis=1)

merged_df.iloc[1]

table_id                                        1-10015132-16
TQL                           What clu was in toronto 1995-96
SQL         SELECT  Years in Toronto FROM table_1_10015132...
db_id                                     table_1_10015132_16
fileName                                                  TBA
filePath                                                  TBA
result                                          Not available
id                                              1-10015132-16
header      [Player, No., Nationality, Position, Years in ...
types                    [text, text, text, text, text, text]
name                                        table_10015132_16
Name: 1, dtype: object

In [177]:
merged_df.columns

Index(['table_id', 'TQL', 'SQL', 'db_id', 'fileName', 'filePath', 'result',
       'id', 'header', 'types', 'name'],
      dtype='object')

In [178]:
data_df = merged_df[['db_id','TQL','SQL','fileName','filePath','result']]

In [179]:
data_df.to_csv('WikiSQL_test.csv',index=False)

In [180]:
schema_df = merged_df[['table_id','db_id','header','types']]

In [181]:
schema_df.rename(columns={'table_id':'table_name','db_id':'table_name_original','header':'column_list','types':'column_datatypes'},inplace=True)

In [182]:
schema_df.columns

Index(['table_name', 'table_name_original', 'column_list', 'column_datatypes'], dtype='object')

In [183]:
# Add 'primary_key' column with values 'NO_PRIMARY_KEY'
schema_df['primary_key'] = 'NO_PRIMARY_KEY'

# Add 'column_list_original' column with values same as 'column_list'
schema_df['column_list_original'] = schema_df['column_list']

# Add 'foreign_keys' column with empty lists
schema_df['foreign_keys'] = [[] for _ in range(len(schema_df))]

In [184]:
schema_df.to_csv('WikiSQL_test_schema.csv',index=False)